In [2]:
import pandas as pd

In [3]:
data=pd.DataFrame([1,4,3,2,5,2,3,])

In [6]:
abs(data.loc[:3].std()-data.std())/data.std()

0    0.040285
dtype: float64

In [8]:
data[1]=[2,3,4,5,2,4,4]

In [10]:
abs(data.loc[:3].std()-data.std())/data.std()

0    0.040285
1    0.138550
dtype: float64

In [2]:
#from portfolio import PortfolioGenerator
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
weights = pd.read_csv('weights.csv')
feature_coefs = pd.read_csv('feature_coefs.csv')
index1_coefs = pd.read_csv('index1.csv')
index2_coefs = pd.read_csv('index2.csv')
index1_coefs= index1_coefs.drop('Unnamed: 0',axis=1)
index2_coefs= index2_coefs.drop('Unnamed: 0',axis=1)


In [4]:
MAX_LOOKBACK = 50

class PortfolioGenerator(object):

    def __init__(self):
        pass

    def read_stock_data(self):
        '''
        Description:
            Reads in simulated stock data from stock_data.csv
        Returns:
            stock_df (DataFrame): standardized ticker/factor data in pandas df
        Raises:
            AssertionError: ticker_data.csv/factor_data.csv has an invalid format
        '''
        ticker_df = pd.read_csv('stock_data/ticker_data.csv')
        factor_df = pd.read_csv('stock_data/factor_data.csv')
        assert 'timestep' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'ticker' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'returns' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'timestep' in factor_df.columns, "factor_data.csv has an invalid format"
        ticker_df.set_index('timestep', inplace=True)
        factor_df.set_index('timestep', inplace=True)
        stock_df = ticker_df.join(factor_df, how='left')
        return stock_df

    def build_signal(self, stock_features):
        '''
        Description:
            Using stock features, generate a buy/sell stock
            signal (suggested from -100 to 100 where -100 is strong
            sell signal) for the stock on a given day.
        Args:
            hist_features (DataFrame) - dataframe of historical features
        Return:
            signal (pandas Series) - key = ticker, value = signal
        Raises:
            NotImplementedError - throws error if the function is not implemented
        '''
        raise NotImplementedError("build_signal must be implemented")

    def simulate_portfolio(self):
        '''
        Description:
            Simulates performance of the portfolio on historical data
        Return:
            sharpe (int) - sharpe ratio for the portfolio
        '''
        daily_returns = []
        stock_df = self.read_stock_data()
        for idx in stock_df.index.unique():
            print("timestep", idx)
            if idx < MAX_LOOKBACK:
                continue
            stock_features = stock_df.loc[idx-MAX_LOOKBACK:idx-1]
            returns = stock_df.loc[idx:idx].set_index('ticker')['returns']
            signal = self.build_signal(stock_features)
            signal_return = returns *signal
            daily_returns.append(np.mean(signal_return))
            #print(daily_returns[idx-50])
        sharpe_ratio = np.sqrt(252) * (np.mean(daily_returns) / np.std(daily_returns))
        #pd.DataFrame(daily_returns,columns=['a']).to_csv('dailyreturn_market.csv')
        
        return sharpe_ratio

In [32]:

class SampleStrategy(PortfolioGenerator):
    def __init__(self):
        pass

    def build_signal(self, stock_features):
        return self.momentum(stock_features)

    def momentum(self, stock_features):
        #estimating the big_ix Small_ix and rf
        poly = preprocessing.PolynomialFeatures(4)
        transformed_features = pd.DataFrame(poly.fit_transform(stock_features[['VIX','COPP','3M_R', 'US_TRY','BIG_IX','SMALL_IX', 'SENTI','TEMP',	'RAIN',	'OIL']].iloc[-1].reshape(1,-1)))
        transformed_features = transformed_features.fillna(0)
        #print(transformed_features.shape)
        #print(index1_coefs.shape)
        ea = model1.predict(transformed_features)+stock_features['3M_R'].iloc[-1]
        eb = model2.predict(transformed_features)+stock_features['3M_R'].iloc[-1]
        #print(index1,index2)
        #ea=my_predict[0]   
        #eb=my_predict[1]
        #print(ea)
        if abs(ea-eb)>1e-10:
            lambda1=(0.11-eb)/(ea-eb)
        else:
            lambda1=1/2
        my_weights=np.zeros(1000)
        my_weights[:500]=lambda1
        my_weights[500:]=(1-lambda1)
        #print(type(stock_features[['VIX','COPP','3M_R', 'US_TRY','BIG_IX','SMALL_IX', 'SENTI','TEMP',	'RAIN',	'OIL']].iloc[-1].reshape(1,-1)))
        return my_weights
        #return 1


if __name__ == "__main__":
    portfolio = SampleStrategy()
    sharpe = portfolio.simulate_portfolio()
    print("*** Strategy Sharpe is {} ***".format(sharpe))

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 5
timestep 6
timestep 7
timestep 8
timestep 9
timestep 10
timestep 11
timestep 12
timestep 13
timestep 14
timestep 15
timestep 16
timestep 17
timestep 18
timestep 19
timestep 20
timestep 21
timestep 22
timestep 23
timestep 24
timestep 25
timestep 26
timestep 27
timestep 28
timestep 29
timestep 30
timestep 31
timestep 32
timestep 33
timestep 34
timestep 35
timestep 36
timestep 37
timestep 38
timestep 39
timestep 40
timestep 41
timestep 42
timestep 43
timestep 44
timestep 45
timestep 46
timestep 47
timestep 48
timestep 49
timestep 50
<class 'numpy.ndarray'>
timestep 51
<class 'numpy.ndarray'>
timestep 52
<class 'numpy.ndarray'>
timestep 53
<class 'numpy.ndarray'>
timestep 54
<class 'numpy.ndarray'>
timestep 55


C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if sys.path[0] == '':
C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


<class 'numpy.ndarray'>
timestep 56
<class 'numpy.ndarray'>
timestep 57
<class 'numpy.ndarray'>
timestep 58
<class 'numpy.ndarray'>
timestep 59
<class 'numpy.ndarray'>
timestep 60
<class 'numpy.ndarray'>
timestep 61
<class 'numpy.ndarray'>
timestep 62
<class 'numpy.ndarray'>
timestep 63
<class 'numpy.ndarray'>
timestep 64
<class 'numpy.ndarray'>
timestep 65
<class 'numpy.ndarray'>
timestep 66
<class 'numpy.ndarray'>
timestep 67
<class 'numpy.ndarray'>
timestep 68
<class 'numpy.ndarray'>
timestep 69
<class 'numpy.ndarray'>
timestep 70
<class 'numpy.ndarray'>
timestep 71
<class 'numpy.ndarray'>
timestep 72
<class 'numpy.ndarray'>
timestep 73
<class 'numpy.ndarray'>
timestep 74
<class 'numpy.ndarray'>
timestep 75
<class 'numpy.ndarray'>
timestep 76
<class 'numpy.ndarray'>
timestep 77
<class 'numpy.ndarray'>
timestep 78
<class 'numpy.ndarray'>
timestep 79
<class 'numpy.ndarray'>
timestep 80
<class 'numpy.ndarray'>
timestep 81
<class 'numpy.ndarray'>
timestep 82
<class 'numpy.ndarray'>
time

KeyboardInterrupt: 

In [ ]:
def my_returns(features):
        poly = preprocessing.PolynomialFeatures(4)
        transformed_features = pd.DataFrame(poly.fit_transform(features))
        transformed_features = transformed_features.fillna(0)
        #print(transformed_features.shape)
        #print(index1_coefs.shape)
        index1 = np.dot(transformed_features,index1_coefs)[0]+stock_features['3M_R'].iloc[-1]
        index2 = np.dot(transformed_features,index2_coefs)[0]+stock_features['3M_R'].iloc[-1]

In [7]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_selection,ensemble, linear_model, cross_validation, decomposition, cluster,model_selection
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import average_precision_score
import pickle

C:\Users\bruce\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\bruce\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [8]:
df1 = pd.DataFrame.from_csv('stock_data/factor_data.csv',index_col=10)
df2 = pd.DataFrame.from_csv('stock_data/ticker_data.csv',index_col=['timestep','ticker'])
df2 = df2.unstack()
df2 = df2.fillna(0)
clean_upcols = {'industry':{'TECH':0,'AGRICULTURE':1,'FINANCE':2,'CONSUMER':3,'OTHER':4}}
df2.replace(clean_upcols,inplace=True)
df2 = df2.drop(['index'],axis=1)


C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
C:\Users\bruce\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [16]:
index1=df2['returns'].loc[:,:500].mean(1)
index2=df2['returns'].loc[:,500:].mean(1)

In [18]:
poly = preprocessing.PolynomialFeatures(degree=4)
factors = pd.DataFrame(poly.fit_transform(df1))
factors = factors.shift(1)
factors = factors.fillna(0)
x = factors
y1 = index1-df1['3M_R'].shift(1).fillna(method='bfill')
y2 = index2-df1['3M_R'].shift(1).fillna(method='bfill')
model1 = linear_model.ElasticNet()
model2 = linear_model.ElasticNet()
model1.fit(x,y1)
model2.fit(x,y2)
pred1 = model1.predict(x)
pred2 = model2.predict(x)
print((y1/pred1).mean(),(y2/pred2).mean())
#index1_coefs = model.coef_
pickle.dump(model1,open('index1.p','wb'))
pickle.dump(model2,open('index2.p','wb'))

C:\Users\bruce\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.9996886225667847 0.9996891806217687


In [19]:
model1 = pickle.load(open('index1.p','rb'))
model2 = pickle.load(open('index2.p','rb'))

In [20]:
model1

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [29]:
df1.loc[0]['3M_R']

0.06

In [35]:
l=[]
for i in range(50,300):
    poly = preprocessing.PolynomialFeatures(4)
    transformed_features = pd.DataFrame(poly.fit_transform(np.array(df1.loc[i]))
    transformed_features = transformed_features.fillna(0)
    #print(my_predict)
    ea = model1.predict(transformed_features)+df1.loc[i]['3M_R']
    eb = model2.predict(transformed_features)+df1.loc[i]['3M_R']
    #print(ea)
    if abs(ea-eb)>1e-10:
        lambda1=(5e-2-eb)/(ea-eb)
    else:
        lambda1=1/2
    l.append(lambda1*index1[i+1]+(1-lambda1)*index2[i+1])

SyntaxError: invalid syntax (<ipython-input-35-b4f1e1d75561>, line 5)

In [ ]:
sqrt(252)*pd.DataFrame(l).mean()/pd.DataFrame(l).std()